<a href="https://colab.research.google.com/github/batulrokadia/CODSOFT/blob/main/Titanic_survival_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

importing the required libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

Data processing

In [2]:
#load dataset to Pandas dataframe
df=pd.read_csv('/content/Titanic-Dataset.csv')

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
#checking how many null values we have
df.isnull().sum()

handling the null values

In [ ]:
#dropping the cabin column as it has majority null values
df=df.drop(columns='Cabin', axis=1)

In [ ]:
#filling the null values in age column with the mean of ages
df['Age'].fillna(df['Age'].mean(), inplace=True)

In [3]:
#filling null values in embarked column with the mode
df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)

In [ ]:
#to find number of people who survived
df['Survived'].value_counts()

data visualisation

In [ ]:
sns.set()

In [ ]:
#countplot for survived column
sns.countplot(x='Survived', data=df)


In [ ]:
#survivors based on gender
sns.countplot(x='Sex', hue='Survived', data=df)

In [ ]:
#no. of people embarked from the 3 different ports
sns.countplot(x='Embarked', data=df)

In [ ]:
#survivors based on embarked town
sns.countplot(x='Survived', hue='Embarked', data=df)

In [ ]:
#survivors based on pclass
sns.countplot(x='Pclass', hue='Survived', data=df)

In [ ]:
plt.hist(df['Age'])
plt.show()

Convert categorical columns to numerical

In [ ]:
#converting categorical columns to numerical
df.replace({'Sex':{'male':0,'female':1}, 'Embarked':{'S':0,'C':1,'Q':2}}, inplace=True)
df.head()

Decide the columns required for training the model

In [ ]:
#dropping the columns which we do not need
y=df['Survived']
x=df.drop(columns=['PassengerId', 'Name','Ticket','Survived'], axis=1)
print(x)
print(y)


Splitting data into training data and test data

In [ ]:
x_train, x_test, y_train, y_test= train_test_split(x, y, test_size=0.2, random_state=2)
print(x.shape, x_train.shape, x_test.shape)

Model Training

Logistic Regression

In [ ]:
model= LogisticRegression()

In [ ]:
#training the model with training data
model.fit(x_train,y_train)

Evaluating the model's accuracy

In [ ]:
training_model= model.predict(x_train)
print(training_model)

In [ ]:
accuracy_of_training=accuracy_score(y_train, training_model)
print("Model's accuracy in percentage is: ", accuracy_of_training*100, '%')

In [ ]:
testing_model=model.predict(x_test)
print(testing_model)

In [ ]:
accuracy_of_testing=accuracy_score(testing_model, y_test)
print("Model's accuracy in percentage is: ", accuracy_of_testing*100, '%')

Using the model to make a prediction

In [ ]:
result=model.predict([[1,0,3,1,2,7.0,0]])
if(result==0):
  print("Not survived")
else:
  print("survived")